In [1]:
from pathlib import Path

In [2]:
ROOT_DIR = Path("..").resolve()
VIDEOS_DIR = ROOT_DIR / "data" / "lsa64_preprocessed_hand_videos"

In [3]:
from mat73 import loadmat

data_dict = loadmat(ROOT_DIR / "data" / "lsa64_positions_preprocessed.mat")

In [4]:
import pandas as pd

df = pd.DataFrame(data_dict["db"])

df.head()

,class,hand_left_exist_in_frame,hand_positions_left,hand_positions_right,hand_right_exist_in_frame,repetition,subject
0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-99.99, 99.99], [-99.99, 99.99], [-99.99, 99...","[[-3.089927371765774, -2.05708011802088], [-3....","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0
1,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-99.99, 99.99], [-99.99, 99.99], [-99.99, 99...","[[-3.028544374303751, -2.033803688575319], [-2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",2.0,1.0
2,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-99.99, 99.99], [-99.99, 99.99], [-99.99, 99...","[[-3.1987415946205573, -1.6210551984039017], [...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",3.0,1.0
3,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-99.99, 99.99], [-99.99, 99.99], [-99.99, 99...","[[-2.5983912437632046, -1.8498795343192342], [...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",4.0,1.0
4,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-99.99, 99.99], [-99.99, 99.99], [-99.99, 99...","[[-2.8374232806549053, -1.8745046622597732], [...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",5.0,1.0


In [5]:
def to_str(n: int) -> str:

    s = str(n)

    if n < 100:
        while len(s) < 3:
            s = '0' + s

    return s

In [15]:
for col in ["class", "repetition", "subject"]:
    df[col] = df[col].astype(int)
    df[col] = df[col].transform(to_str)

df = df.drop(
    labels=["hand_left_exist_in_frame", "hand_positions_left", "hand_positions_right", "hand_right_exist_in_frame"],
    axis=1
)

In [16]:
df.head()

,class,repetition,subject
0,001,001,001
1,001,002,001
2,001,003,001
3,001,004,001
4,001,005,001


In [8]:
df.shape

(3200, 7)

Here we save the videos' metadata in a more suitable format

In [17]:
df.to_feather("metadata.feather")

The following cell reads all frames for a particular video.

The shape of the resulting object (`frames`) is `[<number of frames in video> x 128 x 128 x 3]`.

This represents just one sample in our dataset.

In [18]:
def get_metadata_from_filename(filename: str):
    """CLASS_SUBJECT_REPETITION_hand"""
    filename = filename.split(".")[0]

    klass, subject, repetition , _ = filename.split("_")

    return klass, subject, repetition


In [23]:
import os
from pathlib import Path

targets = []
subjects = []
repetitions = []
files = []

for file in os.listdir(VIDEOS_DIR):
    if "left" in file:
        continue
    
    target, subject, repetition = get_metadata_from_filename(file)

    targets.append(target)
    subjects.append(subject)
    repetitions.append(repetition)
    files.append((VIDEOS_DIR / file).resolve())

    video = cv.VideoCapture(str(VIDEOS_DIR / file))

    frames = []

    while True:
        ret, frame = video.read()

        if ret:
            frames.append(frame)
        else:
            break

frames = np.array(frames)

In [25]:
metadata = pd.DataFrame(
    data={
        "target": targets,
        "subject": subjects,
        "repetition": repetitions,
        "file": files,
    }
)

metadata.head()

,target,subject,repetition,file
0,064,002,001,/Users/facundopalavecino/Documents/DEV/ecd-tra...
1,035,010,003,/Users/facundopalavecino/Documents/DEV/ecd-tra...
2,033,009,003,/Users/facundopalavecino/Documents/DEV/ecd-tra...
3,026,002,004,/Users/facundopalavecino/Documents/DEV/ecd-tra...
4,023,005,002,/Users/facundopalavecino/Documents/DEV/ecd-tra...


In [28]:
frames.shape

(81, 128, 128, 3)

In [13]:
import cv2 as cv
import numpy as np

video_name = "053_001_001"

input_video_path = VIDEOS_DIR / f"{video_name}_right.avi"

video = cv.VideoCapture(str(input_video_path))

frames = []

while True:
    ret, frame = video.read()

    if ret:
        frames.append(frame)
    else:
        break

frames = np.array(frames)

frames.shape

(67, 128, 128, 3)

In [21]:
metadata.shape

(3200, 4)